In [3]:
%matplotlib inline

import pandas as pd

customers = pd.read_csv("../Mall_Customers.csv")
customers.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [4]:
customers.describe()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000,200.000000
mean,100.500000,38.850000,60.560000,50.200000
std,57.879185,13.969007,26.264721,25.823522
min,1.000000,18.000000,15.000000,1.000000
25%,50.750000,28.750000,41.500000,34.750000
50%,100.500000,36.000000,61.500000,50.000000
75%,150.250000,49.000000,78.000000,73.000000
max,200.000000,70.000000,137.000000,99.000000


In [9]:
customers.isnull().sum()

CustomerID                0
Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

In [11]:
customers = pd.get_dummies(customers, drop_first=True)
customers.head()

,CustomerID,Age,Annual Income (k$),Spending Score (1-100),Gender_Male
0,1,19,15,39,1
1,2,21,15,81,1
2,3,20,16,6,0
3,4,23,16,77,0
4,5,31,17,40,0


In [12]:
clean_customers = customers.drop('CustomerID', axis=1)

# K-means

In [14]:
from sklearn.cluster import KMeans

customer_clusters = KMeans(n_clusters=3, random_state=5)

customer_clusters.fit(clean_customers)
customer_clusters.cluster_centers_

array([[40.39473684, 87.        , 18.63157895,  0.52631579],
       [32.69230769, 86.53846154, 82.12820513,  0.46153846],
       [40.32520325, 44.15447154, 49.82926829,  0.40650407]])

In [16]:
clean_customers['Cluster'] = customer_clusters.predict(clean_customers)
clean_customers.head()

,Age,Annual Income (k$),Spending Score (1-100),Gender_Male,Cluster
0,19,15,39,1,2
1,21,15,81,1,2
2,20,16,6,0,2
3,23,16,77,0,2
4,31,17,40,0,2


In [17]:
clean_customers['Cluster'].value_counts()

2    123
1     39
0     38
Name: Cluster, dtype: int64

# Hierarchical Clustering